## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
load_dotenv()           ## Loading environment variables

groq_api_key = os.getenv("GROQ_API_KEY")

/Users/hyashwanth/Desktop/Learning_Gen_AI/Generative-AI-Learnings/Lang_Chain/1-Langchain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ChatGroq(model ="llama-3.1-8b-instant", groq_api_key=groq_api_key)
model.invoke([HumanMessage(content = "Hi, Myname is Yash and I am a Staff Data Scientiest")])

AIMessage(content="Hello Yash. It's nice to meet you. As a Staff Data Scientist, I'm sure you have a wide range of responsibilities and expertise in data analysis, machine learning, and decision-making. What specific areas of data science are you most interested in or currently working on?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 50, 'total_tokens': 107, 'completion_time': 0.095497931, 'completion_tokens_details': None, 'prompt_time': 0.003413893, 'prompt_tokens_details': None, 'queue_time': 0.051336436, 'total_time': 0.098911824}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c05d0-8323-7e30-92f4-a7bad5cfc2ea-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 50, 'output_tokens': 57, 'total_tokens': 107})

In [3]:
model.invoke(
    [
        HumanMessage(content = "Hi, Myname is Yash and I am a Staff Data Scientiest"),
        AIMessage(content="Nice to meet you, Yash! As a Staff Data Scientist, I'm sure you have a wide range of responsibilities and expertise in the field of data science. What specific areas of data science are you involved with, such as machine learning, deep learning, natural language processing, or something else?"),
        HumanMessage(content="Hey What's my name and what do I do?")
        ])

AIMessage(content='Your name is Yash, and you are a Staff Data Scientist.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 131, 'total_tokens': 146, 'completion_time': 0.019775629, 'completion_tokens_details': None, 'prompt_time': 0.008711587, 'prompt_tokens_details': None, 'queue_time': 0.055613583, 'total_time': 0.028487216}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c05d0-8a66-7df0-ab14-316aa450f996-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 131, 'output_tokens': 15, 'total_tokens': 146})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store ={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [5]:
config = {"configurable":{"session_id":"chat1"}}

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content= "Hi, My name is Yashwanth amd I am a Staff Data Scientist")],
    config=config
)
response.content

"Nice to meet you, Yashwanth. As a Staff Data Scientist, I'm sure you have a wide range of responsibilities and expertise in data science. What specific areas of data science are you most interested in or focused on, such as machine learning, natural language processing, or data visualization?"

In [7]:
with_message_history.invoke(
    [HumanMessage(content= "What is my name?")],
    config=config
)

AIMessage(content='Your name is Yashwanth.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 126, 'total_tokens': 135, 'completion_time': 0.010169307, 'completion_tokens_details': None, 'prompt_time': 0.007907834, 'prompt_tokens_details': None, 'queue_time': 0.055313826, 'total_time': 0.018077141}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c05d0-cb81-75d0-b56f-8da792494702-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 126, 'output_tokens': 9, 'total_tokens': 135})

In [8]:
## Changing th session id and lets check model can remember name
config_1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content= "What is my name?")],
    config=config_1
)
response.content

"I don't have any information about your name. I'm a large language model, I don't have the ability to know or remember your personal details unless you choose to share them with me. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I can try to address you by it, but it's not necessary for our conversation."

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content= "Hey, My Name is John")],
    config=config_1
)
response.content

"Hi John. It's nice to make your acquaintance. Is there something I can help you with or would you like to chat about something in particular?"

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content= "What is my name?")],
    config=config_1
)
response.content

'Your name is John.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [12]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Yashwanth!")]})

AIMessage(content="Hello Yashwanth, nice to meet you. I'm here to help and assist you with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 60, 'total_tokens': 99, 'completion_time': 0.074933178, 'completion_tokens_details': None, 'prompt_time': 0.004270266, 'prompt_tokens_details': None, 'queue_time': 0.057647064, 'total_time': 0.079203444}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c05d2-a9bb-72d3-9b38-a3dc156dfb1d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 60, 'output_tokens': 39, 'total_tokens': 99})

In [14]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

config ={"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Yashwanth")],
     config=config
)
response.content

"Hello Yashwanth. It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [15]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "you are a helpful assistant. Answer all questions to the best of your ability in {language}"
         ),
         MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [16]:
response = chain.invoke({"messages":[HumanMessage(content="Hi My name is Yashwanth")],
                         "language":"Kannada"})
response.content

'ನಮಸ್ಕಾರ Yashwanth ಅವರಿಗೆ! ನಿಮಗೆ ನಾನು ಸಹಾಯ ಮಾಡಲು ಇಷ್ಟವಾಗಿದೆ. ಯಾವ ಸಮಸ್ಯೆಯನ್ನು ನೀವು ಹೇಳಬಯಸುತ್ತೀರಿ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [18]:
config ={"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I am Yash")],"language":"Hindi"},
    config = config
)
response.content

'नमस्ते यश, मैं आपकी सहायता के लिए तैयार हूँ। आप क्या पूछना चाहते हैं?'

In [19]:
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="What is my name?")],"language":"Hindi"},
    config = config
)
response.content

'आपका नाम यश है!'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [22]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)
response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What ice cream do i like?")],
    "language":"English"
    }
)
response.content

"I'm not aware of any specific information about your preferences, so I couldn't tell you what ice cream you like. If you'd like to share, I'd be happy to chat with you about your favorite flavors!"

In [25]:

response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What math problem did i asked?")],
    "language":"English"
    }
)
response.content

'You asked me to solve the math problem "2 + 2".'

In [31]:
## Lets wrap this with message history
with_message_history = RunnableWithMessageHistory(
chain,
get_session_history,
input_messages_key = "messages"
)
config = {"configurable":{"session_id":"chat6"}}

In [32]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="Whats my name?")],
        "language":"English",
    },
    config=config
)
response.content

"I don't think we've established your name yet. We just started chatting, and I don't have any information about you. Would you like to share your name?"

In [33]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did I ask?")],
        "language":"English",
    },
    config=config
)
response.content

'You asked about the math problem 2 + 2.'